In [24]:
import numpy as np
import math
import random
import uuid 

In [72]:
class Node: 
    def __init__(self, id, latitude, longitude): 
        self.id = id
        self.latitude = latitude
        self.longitude = longitude

    def distance_to(self, second_node):
        '''Haversine formula for geodesic distance
        a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
        c = 2 ⋅ atan2( √a, √(1−a) )
        d = (Earth Radius) ⋅ c
        '''
        R = 6371e3 #m
        lat1 = self.latitude * np.pi/180; #rad
        lat2 = second_node.latitude * np.pi/180; #rad
        delta_lat = (second_node.latitude - self.latitude) * np.pi/180;
        delta_long = (self.longitude - second_node.longitude) * np.pi/180;
        a = (np.sin(delta_lat/2))**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_long/2) * np.sin(delta_long/2)
        c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
        d = R * c #m
        return d

class KNN: 
    def __init__(self, weather_stations, pm_stations, radius):
        self.weather_stations = weather_stations
        self.pm_stations = pm_stations
        self.radius = radius

    def get_neighbours(self, pm_station):
        nearest_weather_stations = list()
        for ws in self.weather_stations:
            dist = pm_station.distance_to(ws)
            if dist <= self.radius: 
                nearest_weather_stations.append({'id':ws.id, 'dist':dist})
        return nearest_weather_stations
    
    def find_corresponding_weather_stations(self, prune_val = None):
        pm_station_neighbours = {}
        for pms in self.pm_stations: 
            nearest_weather_stations = self.get_neighbours(pms)
            if prune_val != None: 
                nearest_weather_stations.sort(key=lambda dic: dic['dist'])
                nearest_weather_stations = nearest_weather_stations[:prune_val]
            pm_station_neighbours[pms.id] = nearest_weather_stations

            distances = [nws['dist'] for nws in nearest_weather_stations]
            distance_weights = [float(i)/sum(distances) for i in distances]
            for i,nws in enumerate(nearest_weather_stations):
                nws['weight'] = distance_weights[i]
        return pm_station_neighbours


            



    



In [68]:
#test with random latitude longitude
def randlatlon1():
    pi = math.pi
    cf = 180.0 / pi  # radians to degrees Correction Factor

    # get a random Gaussian 3D vector:
    gx = random.gauss(0.0, 1.0)
    gy = random.gauss(0.0, 1.0)
    gz = random.gauss(0.0, 1.0)

    # normalize to an equidistributed (x,y,z) point on the unit sphere:
    norm2 = gx*gx + gy*gy + gz*gz
    norm1 = 1.0 / math.sqrt(norm2)
    x = gx * norm1
    y = gy * norm1
    z = gz * norm1

    radLat = math.asin(z)      # latitude  in radians
    radLon = math.atan2(y,x)   # longitude in radians

    return (round(cf*radLat, 5), round(cf*radLon, 5))

In [73]:
weather_stations = []
for _ in range(100): 
    info = randlatlon1()
    id = uuid.uuid4().hex[:6].upper()
    node = Node(id = id, latitude = info[0], longitude = info[1])
    weather_stations.append(node)

pm_stations = []
for _ in range(10): 
    info = randlatlon1()
    id = uuid.uuid4().hex[:6].upper()
    node = Node(id = id, latitude = info[0], longitude = info[1])
    pm_stations.append(node)


In [74]:
knn = KNN(weather_stations, pm_stations, radius = 10e6)

In [75]:
pm_station_neighbours = knn.find_corresponding_weather_stations(prune_val = 5)

In [76]:
pm_station_neighbours

{'0DA37A': [{'dist': 1942151.2817053495,
   'id': '48519E',
   'weight': 0.1648095658318121},
  {'dist': 2051892.9389842923, 'id': '543B21', 'weight': 0.17412216421700322},
  {'dist': 2486772.2481413884, 'id': '639318', 'weight': 0.21102571071544382},
  {'dist': 2650552.6672399673, 'id': '14EA20', 'weight': 0.22492399969923893},
  {'dist': 2652845.402102786, 'id': 'DC64F8', 'weight': 0.2251185595365019}],
 '11594D': [{'dist': 573695.9764104239,
   'id': '3D66E5',
   'weight': 0.06748703806064811},
  {'dist': 970827.7472335892, 'id': '4AEB2C', 'weight': 0.11420384981228195},
  {'dist': 1883088.0871693098, 'id': '62AEF5', 'weight': 0.2215180908283588},
  {'dist': 2486423.553448188, 'id': 'ED3776', 'weight': 0.29249178639245704},
  {'dist': 2586796.6902563423, 'id': '09C398', 'weight': 0.304299234906254}],
 '29F915': [{'dist': 1971119.579505225,
   'id': '5E1E23',
   'weight': 0.15092261368278817},
  {'dist': 2188789.7619781066, 'id': 'FD75D4', 'weight': 0.16758895559384707},
  {'dist': 2